In [ ]:
import os
import boto3
from dotenv import load_dotenv

In [ ]:
load_dotenv()

In [ ]:
os.getenv('REgION_NAME')

In [ ]:
path = os.getenv('FILE_PATH')

In [ ]:
with open(path + "\support_logs_2025-07-01.log", encoding="utf-8") as data:
    content = data.read()

In [ ]:
#print(content)

In [ ]:
entries = [entry.strip() for entry in content.split("---") if entry.strip()]
entries 

In [ ]:
import re

In [ ]:
pattern = re.compile(
    r'(?P<timestamp>\d{4}-\d{2}-\d{2}\s+\d{2}:\d{2}:\d{2}).*?'
    r'\[(?P<loglevel>[A-Z0-9]+)\].*?'
    r'(?P<component>[\w\.]+)\s+-.*?'
    r'TicketID=(?P<ticketid>\S+).*?'
    r'SessionID=(?P<sessionid>\S+).*?'
    r'IP=(?P<ip>[\d\.]+).*?'
    r'ResponseTime=(?P<responsetime>\d+ms).*?'
    r'CPU=(?P<cpu>[\d\.]+%).*?'
    r'EventType=(?P<eventtype>\S+).*?'
    r'Error=(?P<error>\S+).*?'
    r'UserAgent="(?P<useragent>[^"]+)".*?'
    r'Message="(?P<message>[^"]+)".*?'
    r'Debug="(?P<debug>[^"]+)".*?'
    r'TraceID=(?P<traceid>\S+)',
    re.DOTALL
)

In [ ]:
match = pattern.search(entries[0])

In [ ]:
if match:
    data = match.groupdict()
    print(data)
else:
    print("No match found")

In [ ]:
#Extract the Structured Data
parsed_data = []
for entry in entries:
    match = pattern.search(entry)
    if match:
        parsed_data.append(match.groupdict())

In [ ]:
parsed_data

### Data Cleaning

In [ ]:
#dataframe
import pandas as pd

df = pd.DataFrame(parsed_data)
df.head()

In [ ]:
df.info()

In [ ]:
#fix responsetime column
df['responsetime'] = df['responsetime'].str.replace('ms', '')
#fix cpu column
df['cpu'] = df['cpu'].str.replace('%', '')
df.head(1)

In [ ]:
df = df.astype({
    'responsetime': 'int',
    'cpu': 'float'
})

In [ ]:
df.info()

In [ ]:
# fix Errr column and timestamp column
df['error'] = df['error'].str.lower().map({'true': True, 'false': False})
df['timestamp'] = pd.to_datetime(df['timestamp'],format='%Y-%m-%d %H:%M:%S', errors='coerce')

In [ ]:
df.info()

In [ ]:
df.describe()

In [ ]:
df = df[df['responsetime'] >= 0]
df.describe()

In [ ]:
df['loglevel'].value_counts()

In [ ]:
df['loglevel'] = df['loglevel'].replace('INF0', 'INFO')

In [ ]:
df['loglevel'].value_counts()

In [ ]:
len(df[df.duplicated()])

In [ ]:
df.drop_duplicates(inplace=True)

In [ ]:
df.duplicated().sum()

In [ ]:
df.shape

In [ ]:
df